## Load Data, Preprocessing

The dataset we have to feed consist of 'Tweets' according to the kaggle guide,
which means that there sould be lots of 'cooloquial expressions','hashtags','links',etc.

Thus, we have to "clean" this data(I mean, "text") before we feed it to our model.

In [1]:
import pandas as pd

train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')
train_len = len(train) # we should record it to split all_data into train,test again

all_data = pd.concat([train,test]) # for preprocessing and tokenizing

In [2]:
import re
import string

# Cleaning Functions
def remove_tag(text):
    tag = re.compile(r'@\S+')
    return tag.sub(r'',text)

def remove_URL(text):
    # http:... / https:... / www... 
    url = re.compile(r'https?://\S+|www\.\S+')
    return re.sub(url,'',text)

def remove_html(text):
    # < > / ( )
    html = re.compile(r'<[^>]+>|\([^)]+\)')
    return html.sub(r'',text)

def remove_punct(text):
    # ['!','"','$','%','&',"'",'(',')','*',
    # '+',',','-','.','/',':',';','<','=',
    # '>','?','@','[','\\',']','^','_','`',
    # '{','|','}','~']
    punctuations = list(string.punctuation)
    table = str.maketrans('', '', ''.join(punctuations))
    return text.translate(table)



In [3]:
'''
If you want to correct some misspelled words, you can try below also.
But I supposed that there must be some other misspelled also in "test set".

I thought, if I corrected these words in training process, there would be a chacne that
the model couldn't bear some "noises"(misspelled words).

So, In my opinion, this task of correction must be accompanied with "result analysis".
'''
# from spellchecker import SpellChecker

# spell = SpellChecker()
# def correct_spellings(text):
#     corrected_text = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_text.append(spell.correction(word))
#         else:
#             corrected_text.append(word)
#     return " ".join(corrected_text)

In [3]:
'''
Nltk is used to remove stopwords and punktuations.

Stopwords are words that are used frequently, but have no 'semantic' meaning,
which could affact model training (I mean, negatively).

And punctuations like '.', ',' also seems they don't have meaningful effect of interpreting sentences.
'''
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chanmuzi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/chanmuzi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
all_data['cleaned'] = all_data['text'].apply(lambda x:remove_tag(x))
all_data['cleaned'] = all_data['cleaned'].apply(lambda x: remove_URL(x))
all_data['cleaned'] = all_data['cleaned'].apply(lambda x: remove_html(x))
all_data['cleaned'] = all_data['cleaned'].apply(lambda x: remove_punct(x))
all_data['cleaned'] = all_data['cleaned'].apply(lambda x: x.lower()) # lowering
all_data['cleaned'] = all_data['cleaned'].apply(lambda x: word_tokenize(x)) # split sentence into words list
# exclude stop words and make them a sentence again
all_data['cleaned'] = all_data['cleaned'].apply(lambda x: ' '.join([word for word in x if word not in stop]))

## Dataset, DataLoader

In [5]:
# Remember we have combined train and test set into one "all_data"
train_data,test_data = all_data[:train_len],all_data[train_len:]

In [43]:
from torch.utils.data import Dataset
import torch

class TweetDataset(Dataset):
    def __init__(self,df,label,tokenizer):
        self.df = df # Pandas.DataFrame
        self.label = label # True: train,valid / False: test
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df) # number of samples

    def __getitem__(self,idx):
        text = self.df.loc[idx]['text'] # extracting text from each row

        encoded_dict = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=84, # given to the max_length of tokenized text
            return_tensors='pt', # PyTorch
            return_attention_mask=True, # We should put it into the model
        )
        '''
        The model BERT has two positional(mandatory) arguments,
        "input_ids" and "attention_mask".

        In the process of train/valid, we already have labels(answers),
        but in case of test, we don't.
        '''
        if self.label:
            labels = self.df.loc[idx]['target']
            # [batch,1,max_len(84)] -> [batch,max_len]
            return {'input_ids':encoded_dict['input_ids'].squeeze(),
                    'attention_mask':encoded_dict['attention_mask'].squeeze(),
                    # Our loss_fn wants it to be a "long tensor", so it will be changed
                    'labels':torch.tensor(labels,dtype=torch.int).unsqueeze(dim=0)}
        else:
            # [batch,1,max_len(84)] -> [batch,max_len]
            return {'input_ids':encoded_dict['input_ids'].squeeze(),
                    'attention_mask':encoded_dict['attention_mask'].squeeze()}

In [44]:
from transformers import BertTokenizer
model_name = 'bert-base-uncased' # If possible, use "bert-large-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

train_dataset = TweetDataset(train_data,True,tokenizer)
test_dataset = TweetDataset(test_data,False,tokenizer)

In [45]:
from torch.utils.data import random_split

train_size = int(0.8 * len(train_dataset)) # train:valid = 8:2
valid_size = len(train_dataset) - train_size

train_dataset,valid_dataset = random_split(train_dataset,[train_size,valid_size])

print(f'{len(train_dataset)} train samples')
print(f'{len(valid_dataset)} valid samples')
print(f'{len(test_dataset)} test samples')

6090 train samples
1523 valid samples
3263 test samples


In [46]:
from torch.utils.data import DataLoader

'''
DataLoader in torch is useful tools for model to get data "in Batch".

So, the steps we have to take are like this, 
"Load CSV file -> make Dataset(in torch.utils.data) -> make DataLoader"

Please pay attention to the fact that I only "shuffled" train set, not valid/test set.
'''
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
valid_dataloader = DataLoader(valid_dataset,batch_size=32,shuffle=False,pin_memory=True)
test_dataloader = DataLoader(test_dataset,batch_size=1,shuffle=False)

## Model

In [10]:
'''
You can give some info to the model,tokenizer or something
instead of using arguments below.

But, I prefer to use it because it makes experiment more easier.
I mean, there is only one cell I have to change for an experiment
making some differences in settings.
'''

configs = {
    'model_name':'bert-base-uncased',
    'num_labels':2,
    'batch_size':32,
    'epochs':4,
    'learning_rate':5e-6,
}

In [11]:
import numpy as np
import torch
import torch.nn as nn
from transformers import BertForSequenceClassification

# Never Detach Tensor during forward
class TweetsModel(nn.Module):
    '''
    To be honest, under the setting like this, there is no need to inherit.
    It's because I used "BertForSequenceClassification" which has final layer
    that is composed of "hidden size 2" for binary classification.

    So, you can think of this unnecessary inheritance is kind of "practice" for myself :)
    '''
    def __init__(self,model_name):
        super().__init__()
        self.model = BertForSequenceClassification.from_pretrained(model_name)

    def forward(self,input_ids,attention_mask):
        output = self.model(input_ids=input_ids,attention_mask=attention_mask)
        logits = output.logits
        return logits

In [13]:
if torch.cuda.is_available():
    device = 'cuda'
    print('GPU is running on..')
else: 
    device = 'cpu'
    print('CPU is running on..')

'''
You must check your accelerator setting in the right pannel.
'''
model = TweetsModel(configs['model_name']).to(device)

CPU is turning on..


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Tools

In [14]:
# loss function
# (y_pred,y_label)
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss()

In [15]:
# optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                lr=6e-6,
                eps=1e-8,
                no_deprecation_warning=True)

In [16]:
# metric for validation
# f1_score(y_label,y_pred)
from sklearn.metrics import f1_score

metric = f1_score

## Train

In [17]:
import gc,os
from tqdm.auto import tqdm # visualizing tool for progress

# They will be used to pick the best model.pt given to the valid loss
best_model_epoch, valid_loss_values = [],[] 
valid_loss_min = [1] # arbitrary loss I set here

def train(model,device,train_dataloader,valid_dataloader,epochs,loss_fn,optimizer,metric):

    for epoch in range(epochs):
        gc.collect() # memory cleaning
        model.train()

        train_loss = 0
        train_step = 0
        pbar = tqdm(train_dataloader)

        for batch in pbar: # you can also write like "for batch in tqdm(train_dataloader"
            optimizer.zero_grad() # initialize
            train_step += 1

            train_input_ids = batch['input_ids'].to(device)
            train_attention_mask = batch['attention_mask'].to(device)
            train_labels = batch['labels'].squeeze().to(device).long()
            
            # You can refer to the class "TweetsModel" for understand 
            # what would be logits
            logits = model(train_input_ids, train_attention_mask).to(device)
            predictions = torch.argmax(logits, dim=1) # get an index from larger one
            detached_predictions = predictions.detach().cpu().numpy()
            
            loss = loss_fn(logits, train_labels)
            loss.backward() 
            optimizer.step()
            model.zero_grad()

            train_loss += loss.detach().cpu().numpy().item()

            pbar.set_postfix({'train_loss':train_loss/train_step})
        pbar.close()

        with torch.no_grad():
            model.eval()

            valid_loss = 0
            valid_step = 0
            total_valid_score = 0

            y_pred = [] # for getting f1_score that is a metric of the competition
            y_true = []

            pbar = tqdm(valid_dataloader)
            for batch in pbar:
                valid_step += 1

                valid_input_ids = batch['input_ids'].to(device)
                valid_attention_mask = batch['attention_mask'].to(device)
                valid_labels = batch['labels'].squeeze().to(device).long()

                logits = model(valid_input_ids, valid_attention_mask).to(device)
                predictions = torch.argmax(logits, dim=1)
                detached_predictions = predictions.detach().cpu().numpy()
                
                loss = loss_fn(logits, valid_labels)
                valid_loss += loss.detach().cpu().numpy().item()

                y_pred.extend(predictions.cpu().numpy())
                y_true.extend(valid_labels.cpu().numpy())

            valid_loss /= valid_step
            f1 = f1_score(y_true,y_pred)

            print(f'Epoch [{epoch+1}/{epochs}] Score: {f1}')
            print(f'Epoch [{epoch+1}/{epochs}] Valid_loss: {valid_loss}')

            if valid_loss < min(valid_loss_min):
                print('model improved!')
            else:
                print('model not improved')
    
            torch.save(model.state_dict(), f'epoch:{epoch+1}_model.pt')
            print('save checkpoint!')
            valid_loss_min.append(valid_loss)
            print(f'valid_loss_min:{min(valid_loss_min)}')
        
        # Double check your directory
        best_model_epoch.append(f'save/bert-base/epoch:{epoch+1}_model.pt')
        valid_loss_values.append(valid_loss)
        print('='*100)

    select_best_model() # refer to below function
    print('Train/Valid Completed!!')
    del train_dataloader, valid_dataloader # memory cleaning
    gc.collect()

def select_best_model():
    best_model = best_model_epoch[np.array(valid_loss_values).argmin()]
    os.rename(best_model, best_model.split('.pt')[0] + '_best.pt')

In [18]:
if torch.cuda.is_available():
    print('GPU is running on...')
    device = 'cuda'
else:
    print('CPU is running on...')
    device = 'cpu'

CPU is turning on...


In [ ]:
print(f'Before training, files in current directory: {os.listdir()}')

In [ ]:
print('Training Start!')
print('=' * 100)

train(model,
    device,
    train_dataloader,
    valid_dataloader,
    configs['epochs'],
    loss_fn,
    optimizer,
    metric)

del model, train_dataloader, valid_dataloader
gc.collect()

In [ ]:
print(f'After training, files in current directory: {os.listdir()}')

## Inference

In [39]:
def inference(model,test_dataloader):
    all_preds = []
    model.eval()

    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            logits = model(input_ids,attention_mask)
            logits = logits.detach().cpu().numpy()
            all_preds.append(logits)
    
    return all_preds

In [40]:
for filename in os.listdir():
    if 'best.pt' in filename: 
        best_pt = filename
print(f'Best model.pt: {best_pt}')
check_point = torch.load(best_pt)

model = TweetsModel(configs['model_name']).to(device)
model.to(device)
model.load_state_dict(check_point)

predictions = inference(model,test_dataloader)

  0%|          | 0/3263 [00:00<?, ?it/s]

In [82]:
sample = pd.read_csv('./Data/sample_submission.csv')
sample

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [83]:
predictions = np.argmax(predictions,axis=2)
sample['target'] = predictions
sample.head(10)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [84]:
sample.to_csv('submission.csv',index=False,header=True)